In [244]:
import pandas as pd
import numpy as np

#https://machinelearningmastery.com/lasso-regression-with-python/
# use automatically configured the lasso regression algorithm
from numpy import arange
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold

from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error



In [245]:
prod = pd.read_csv('Product.csv')
sales = pd.read_csv('Sales - No Store.csv')

In [246]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634 entries, 0 to 1633
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   UPC                    1634 non-null   int64  
 1   TPO_PRODUCT_NAME       1634 non-null   object 
 2   TPO_PRODUCT_GROUP      1634 non-null   object 
 3   PRODUCT                1634 non-null   object 
 4   SIZE_ML                1634 non-null   float64
 5   COUNT                  1634 non-null   int64  
 6   PACK_TYPE              1634 non-null   object 
 7   PACK_SUBTYPE           1634 non-null   object 
 8   PACK_CATEGORY_SUBTYPE  1634 non-null   object 
 9   TYPE                   1634 non-null   object 
 10  SWEETNER               1634 non-null   object 
 11  FLAVOUR                1631 non-null   object 
 12  CATEGORY               1634 non-null   object 
 13  INNOVATION_TYPE        523 non-null    object 
 14  START_YEAR             281 non-null    float64
 15  STAR

In [247]:
prod = prod[prod['INNOVATION_TYPE']=="LTO"]

In [248]:
def weeks_between(df):
    if df['START_YEAR']==df['END_YEAR']:
        return df['END_WEEK']-df['START_WEEK']
    else:
        return df['END_WEEK']+(52-df['START_WEEK'])

In [249]:
prod['LTO_Duration'] = prod.apply(weeks_between, axis=1)

In [250]:
def flavours(df):
    if df['FLAVOUR']=="CHERRY" or df['FLAVOUR']=="VANILLA" or df['FLAVOUR']=="CRANBERRY":
        return df['FLAVOUR']
    else:
        return "OTHER"

In [251]:
prod['FLAVOUR'] = prod.apply(flavours, axis=1)

In [1]:
#prod[prod['CATEGORY']=="ISOTONICS"]

In [2]:
#prod.head(50)

In [254]:
prod['START_YEARWK'] = prod.apply(lambda x: (str(int(x['START_YEAR']))+str(int(x['START_WEEK'])).zfill(2)), axis=1)

In [255]:
prod['END_YEARWK'] = prod.apply(lambda x: (str(int(x['END_YEAR']))+str(int(x['END_WEEK'])).zfill(2)), axis=1)

In [256]:
prod['CATEGORY_O']=prod['CATEGORY']

In [3]:
#sales.head()

In [258]:
prod = pd.get_dummies(prod, columns=['CATEGORY_O','PRODUCT','PACK_TYPE','FLAVOUR'], prefix="LTO")

In [259]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4160 entries, 0 to 4159
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CATEGORY       4159 non-null   object 
 1   CAL_TPO_YEAR   4159 non-null   float64
 2   CAL_TPO_MONTH  4159 non-null   float64
 3   CAL_TPO_WEEK   4159 non-null   float64
 4   CAL_WEEK_TYPE  2499 non-null   object 
 5   BN_REGION      4159 non-null   object 
 6   sum(SALES)     4159 non-null   float64
 7   percent_deals  4160 non-null   float64
dtypes: float64(5), object(3)
memory usage: 260.1+ KB


In [260]:
sales=sales.dropna(subset=["CATEGORY"])

In [261]:
sales['YEARWK'] = sales.apply(lambda x: (str(int(x['CAL_TPO_YEAR']))+str(int(x['CAL_TPO_WEEK'])).zfill(2)), axis=1)

In [262]:
sales=sales.fillna({'CAL_WEEK_TYPE':"Regular"})

In [263]:
#sales.head()

,CATEGORY,CAL_TPO_YEAR,CAL_TPO_MONTH,CAL_TPO_WEEK,CAL_WEEK_TYPE,BN_REGION,sum(SALES),percent_deals,YEARWK
0,ALTERNATIVE BEVERAGES,2019.0,3.0,9.0,Regular,ONTARIO,2.458018e+05,0.000000,201909
1,ALTERNATIVE BEVERAGES,2019.0,7.0,27.0,Post-holiday,EAST,4.764324e+04,0.603952,201927
2,CARBONATED SOFT DRINKS,2018.0,1.0,4.0,Pre-holiday,EAST,7.210700e+05,0.169159,201804
3,CARBONATED SOFT DRINKS,2018.0,4.0,16.0,Regular,EAST,8.439480e+05,0.000000,201816
4,CARBONATED SOFT DRINKS,2019.0,11.0,45.0,Regular,QUEBEC,1.825419e+06,0.000000,201945


In [264]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 45 to 1620
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   UPC                         66 non-null     int64  
 1   TPO_PRODUCT_NAME            66 non-null     object 
 2   TPO_PRODUCT_GROUP           66 non-null     object 
 3   SIZE_ML                     66 non-null     float64
 4   COUNT                       66 non-null     int64  
 5   PACK_SUBTYPE                66 non-null     object 
 6   PACK_CATEGORY_SUBTYPE       66 non-null     object 
 7   TYPE                        66 non-null     object 
 8   SWEETNER                    66 non-null     object 
 9   CATEGORY                    66 non-null     object 
 10  INNOVATION_TYPE             66 non-null     object 
 11  START_YEAR                  66 non-null     float64
 12  START_WEEK                  66 non-null     float64
 13  END_YEAR                    66 non

In [265]:
prod = prod[['CATEGORY','START_YEARWK','END_YEARWK','LTO_Duration','LTO_ALTERNATIVE BEVERAGES','LTO_CARBONATED SOFT DRINKS',
             'LTO_ISOTONICS', 'LTO_FRUITWATER', 'LTO_VITAMINWATER', 'LTO_A&W', "LTO_BARQ'S", 'LTO_CANADA DRY', 'LTO_CANADA DRY MOTTS', 'LTO_COCA-COLA', 'LTO_FANTA', 'LTO_SPRITE', 'LTO_POWERADE',
             'LTO_FC','LTO_IC','LTO_CHERRY','LTO_CRANBERRY','LTO_OTHER','LTO_VANILLA'
            
            ]]

In [266]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4159 entries, 0 to 4159
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CATEGORY       4159 non-null   object 
 1   CAL_TPO_YEAR   4159 non-null   float64
 2   CAL_TPO_MONTH  4159 non-null   float64
 3   CAL_TPO_WEEK   4159 non-null   float64
 4   CAL_WEEK_TYPE  4159 non-null   object 
 5   BN_REGION      4159 non-null   object 
 6   sum(SALES)     4159 non-null   float64
 7   percent_deals  4159 non-null   float64
 8   YEARWK         4159 non-null   object 
dtypes: float64(5), object(4)
memory usage: 324.9+ KB


In [267]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 45 to 1620
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CATEGORY                    66 non-null     object 
 1   START_YEARWK                66 non-null     object 
 2   END_YEARWK                  66 non-null     object 
 3   LTO_Duration                66 non-null     float64
 4   LTO_ALTERNATIVE BEVERAGES   66 non-null     uint8  
 5   LTO_CARBONATED SOFT DRINKS  66 non-null     uint8  
 6   LTO_ISOTONICS               66 non-null     uint8  
 7   LTO_FRUITWATER              66 non-null     uint8  
 8   LTO_VITAMINWATER            66 non-null     uint8  
 9   LTO_A&W                     66 non-null     uint8  
 10  LTO_BARQ'S                  66 non-null     uint8  
 11  LTO_CANADA DRY              66 non-null     uint8  
 12  LTO_CANADA DRY MOTTS        66 non-null     uint8  
 13  LTO_COCA-COLA               66 non

In [268]:
import pandasql as ps

sqlcode = '''
select sales.*, (sum([LTO_ALTERNATIVE BEVERAGES])+sum([LTO_CARBONATED SOFT DRINKS])+sum([LTO_ISOTONICS])) AS LTOS ,avg(LTO_Duration) as LTO_AVG_WKS,
sum([LTO_ALTERNATIVE BEVERAGES]) AS LTO_ALTS, sum([LTO_CARBONATED SOFT DRINKS]) AS LTO_CARBONATED, 
sum([LTO_ISOTONICS]) AS LTO_ISOS,
sum([LTO_FRUITWATER]) AS LTO_FR,
sum([LTO_VITAMINWATER]) AS LTO_VW,
sum([LTO_A&W]) AS LTO_AW,
sum([LTO_BARQ'S]) AS LTO_BAR,
sum([LTO_CANADA DRY]) AS LTO_CAN,
sum([LTO_CANADA DRY MOTTS]) AS LTO_CDM,
sum([LTO_COCA-COLA]) AS LTO_COK,
sum([LTO_FANTA]) AS LTO_FAN,
sum([LTO_SPRITE]) AS LTO_SPR,
sum([LTO_POWERADE]) AS LTO_POW,
sum([LTO_FC]) AS LTO_FC_PK,
sum([LTO_IC]) AS LTO_IC_PK,
sum([LTO_CHERRY]) AS LTO_CHER,
sum([LTO_CRANBERRY]) AS LTO_CRAN,
sum([LTO_VANILLA]) AS LTO_VAN,
sum([LTO_OTHER]) AS LTO_OTH



from sales left join prod on 
(sales.YEARWK>=prod.START_YEARWK and sales.YEARWK<=prod.END_YEARWK) AND sales.CATEGORY=prod.CATEGORY
group by sales.CATEGORY, sales.CAL_TPO_YEAR, sales.CAL_TPO_MONTH, sales.CAL_TPO_WEEK, sales.CAL_WEEK_TYPE,
sales.BN_REGION, sales.[sum(SALES)],sales.percent_deals
'''

df = ps.sqldf(sqlcode,locals())

In [269]:
df=df.fillna(0)

In [270]:
df[df["LTOS"]>1]

,CATEGORY,CAL_TPO_YEAR,CAL_TPO_MONTH,CAL_TPO_WEEK,CAL_WEEK_TYPE,BN_REGION,sum(SALES),percent_deals,YEARWK,LTOS,...,LTO_COK,LTO_FAN,LTO_SPR,LTO_POW,LTO_FC_PK,LTO_IC_PK,LTO_CHER,LTO_CRAN,LTO_VAN,LTO_OTH
292,ALTERNATIVE BEVERAGES,2019.0,6.0,22.0,Regular,EAST,35585.019798,0.0,201922,4.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0
293,ALTERNATIVE BEVERAGES,2019.0,6.0,22.0,Regular,ONTARIO,246435.849880,0.0,201922,4.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0
294,ALTERNATIVE BEVERAGES,2019.0,6.0,22.0,Regular,QUEBEC,92506.879851,0.0,201922,4.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0
295,ALTERNATIVE BEVERAGES,2019.0,6.0,22.0,Regular,WEST,253107.969649,0.0,201922,4.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0
296,ALTERNATIVE BEVERAGES,2019.0,6.0,23.0,Regular,EAST,38998.639781,0.0,201923,4.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,ISOTONICS,2019.0,7.0,28.0,Regular,WEST,200424.209724,0.0,201928,2.0,...,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0
1983,ISOTONICS,2019.0,7.0,29.0,Regular,EAST,92285.580068,0.0,201929,2.0,...,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0
1984,ISOTONICS,2019.0,7.0,29.0,Regular,ONTARIO,398384.319901,0.0,201929,2.0,...,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0
1985,ISOTONICS,2019.0,7.0,29.0,Regular,QUEBEC,170861.580054,0.0,201929,2.0,...,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0


In [271]:
df = df.rename(columns={"sum(SALES)": "SALES"})

In [272]:
df = pd.get_dummies(df, columns=['BN_REGION'])

In [273]:
df['EAST_LTOS']=df['LTOS']*df['BN_REGION_EAST']
df['QC_LTOS']=df['LTOS']*df['BN_REGION_QUEBEC']
df['WEST_LTOS']=df['LTOS']*df['BN_REGION_WEST']
df['ONTARIO_LTOS']=df['LTOS']*df['BN_REGION_ONTARIO']

Winter – December, January and February.
Spring – March, April and May.
Summer – June, July and August.
Autumn – September, October and November.

In [274]:
def seasons(df):
    if df['CAL_TPO_MONTH']==12 or df['CAL_TPO_MONTH']==1 or df['CAL_TPO_MONTH']==2:
        return "Winter"
    elif df['CAL_TPO_MONTH']==3 or df['CAL_TPO_MONTH']==4 or df['CAL_TPO_MONTH']==5:
        return "Spring"
    elif df['CAL_TPO_MONTH']==6 or df['CAL_TPO_MONTH']==7 or df['CAL_TPO_MONTH']==8:
        return "Summer"
    elif df['CAL_TPO_MONTH']==9 or df['CAL_TPO_MONTH']==10 or df['CAL_TPO_MONTH']==11:
        return "Fall"

In [275]:
df['SEASON'] = df.apply(seasons, axis=1)

In [276]:
df = df[(df["CATEGORY"]=="CARBONATED SOFT DRINKS")|(df["CATEGORY"]=="ISOTONICS")|(df["CATEGORY"]=="ALTERNATIVE BEVERAGES")]

In [277]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1248 entries, 0 to 2078
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CATEGORY           1248 non-null   object 
 1   CAL_TPO_YEAR       1248 non-null   float64
 2   CAL_TPO_MONTH      1248 non-null   float64
 3   CAL_TPO_WEEK       1248 non-null   float64
 4   CAL_WEEK_TYPE      1248 non-null   object 
 5   SALES              1248 non-null   float64
 6   percent_deals      1248 non-null   float64
 7   YEARWK             1248 non-null   object 
 8   LTOS               1248 non-null   float64
 9   LTO_AVG_WKS        1248 non-null   float64
 10  LTO_ALTS           1248 non-null   float64
 11  LTO_CARBONATED     1248 non-null   float64
 12  LTO_ISOS           1248 non-null   float64
 13  LTO_FR             1248 non-null   float64
 14  LTO_VW             1248 non-null   float64
 15  LTO_AW             1248 non-null   float64
 16  LTO_BAR            1248 

In [278]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [279]:
dep_var = ['SALES']

#categorical_vars = ['Pclass', 'Embarked']
categorical_vars = ['CAL_TPO_MONTH', 'CAL_WEEK_TYPE','SEASON', 'CATEGORY']

discrete_vars = ['LTO_ALTS','LTO_CARBONATED','LTO_ISOS','LTO_FR', 'LTO_VW', 'LTO_AW', 'LTO_BAR', 'LTO_CAN', 'LTO_CDM', 'LTO_COK', 'LTO_FAN', 'LTO_SPR', 'LTO_POW', 
                 'BN_REGION_ONTARIO','BN_REGION_EAST','BN_REGION_QUEBEC','BN_REGION_WEST', 'ONTARIO_LTOS','EAST_LTOS','QC_LTOS','WEST_LTOS',
                 'LTO_FC_PK','LTO_IC_PK','LTO_CHER','LTO_CRAN','LTO_VAN','LTO_OTH'
                ]




continuous_vars = ['percent_deals','LTO_AVG_WKS']

In [280]:
cat_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

cat_transformers = [('cat', cat_pipe, categorical_vars)]

num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="median")),
])
    #('scl', StandardScaler())])

num_transformers = [('num', num_pipe, discrete_vars + continuous_vars)]

In [281]:
preprocessor = ColumnTransformer(transformers=cat_transformers + num_transformers)

In [282]:
X = df[categorical_vars+discrete_vars+continuous_vars]
y = df[dep_var]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20,random_state=0)

## Lasso CV - Find Alpha

In [283]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
clf = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1, max_iter = 1000000,tol =5000)

model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', clf)])

model.fit(X_train, y_train)

C:\Users\jdonv\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\jdonv\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
C:\Users\jdonv\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
C:\Users\jdonv\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
C:\Users\jdonv\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:527: Us

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['CAL_TPO_MONTH',
                                                   'CAL_WEEK_TYPE', 'SEASON',
                                                   'CATEGORY']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['LTO_ALTS', 'LTO_CARBONATED',
     

In [284]:
print('alpha: %f' % model['classifier'].alpha_)

alpha: 0.890000


In [285]:
model['preprocessor'].transformers_[0][1]['onehot']\
                   .get_feature_names(categorical_vars)

array(['CAL_TPO_MONTH_1.0', 'CAL_TPO_MONTH_2.0', 'CAL_TPO_MONTH_3.0',
       'CAL_TPO_MONTH_4.0', 'CAL_TPO_MONTH_5.0', 'CAL_TPO_MONTH_6.0',
       'CAL_TPO_MONTH_7.0', 'CAL_TPO_MONTH_8.0', 'CAL_TPO_MONTH_9.0',
       'CAL_TPO_MONTH_10.0', 'CAL_TPO_MONTH_11.0', 'CAL_TPO_MONTH_12.0',
       'CAL_WEEK_TYPE_Holiday', 'CAL_WEEK_TYPE_Post-holiday',
       'CAL_WEEK_TYPE_Pre-holiday', 'CAL_WEEK_TYPE_Regular',
       'SEASON_Fall', 'SEASON_Spring', 'SEASON_Summer', 'SEASON_Winter',
       'CATEGORY_ALTERNATIVE BEVERAGES',
       'CATEGORY_CARBONATED SOFT DRINKS', 'CATEGORY_ISOTONICS'],
      dtype=object)

In [286]:
feature_names = ['CAL_TPO_MONTH_1.0', 'CAL_TPO_MONTH_2.0', 'CAL_TPO_MONTH_3.0',
                 'CAL_TPO_MONTH_4.0', 'CAL_TPO_MONTH_5.0', 'CAL_TPO_MONTH_6.0', 
                 'CAL_TPO_MONTH_7.0', 'CAL_TPO_MONTH_8.0', 'CAL_TPO_MONTH_9.0',
                 'CAL_TPO_MONTH_10.0', 'CAL_TPO_MONTH_11.0', 'CAL_TPO_MONTH_12.0',
                 'CAL_WEEK_TYPE_Holiday', 'CAL_WEEK_TYPE_Post-holiday',
                 'CAL_WEEK_TYPE_Pre-holiday', 'CAL_WEEK_TYPE_Regular',
                 'SEASON_Fall', 'SEASON_Spring', 'SEASON_Summer', 'SEASON_Winter', 'CATEGORY_ALTERNATIVE BEVERAGES',
                 'CATEGORY_CARBONATED SOFT DRINKS', 'CATEGORY_ISOTONICS',
                 'LTO_ALTS','LTO_CARBONATED','LTO_ISOS','LTO_FR', 'LTO_VW', 'LTO_AW', 'LTO_BAR', 'LTO_CAN', 'LTO_CDM', 'LTO_COK', 'LTO_FAN', 'LTO_SPR', 'LTO_POW',
                 'BN_REGION_ONTARIO','BN_REGION_EAST','BN_REGION_QUEBEC','BN_REGION_WEST',
                 'ONTARIO_LTOS','EAST_LTOS','QC_LTOS','WEST_LTOS','LTO_FC_PK','LTO_IC_PK','LTO_CHER','LTO_CRAN','LTO_VAN','LTO_OTH', 'percent_deals','LTO_AVG_WKS',]

In [287]:
coeffs = []
for i in range(0, len(feature_names)):
    coeffs.append((model['classifier'].coef_[i],feature_names[i]))

In [4]:
#coeffs

In [289]:
train_score=model.score(X_test,y_test)

train_score

0.8536482458374289

## Lasso CV - Train/Test Split

In [290]:
lr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', Lasso(max_iter = 100000, alpha=0.89))])

lr.fit(X_train, y_train)

C:\Users\jdonv\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 57373170746334.375, tolerance: 306184204201.6733
  positive)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['CAL_TPO_MONTH',
                                                   'CAL_WEEK_TYPE', 'SEASON',
                                                   'CATEGORY']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['LTO_ALTS', 'LTO_CARBONATED',
     

In [291]:
train_score=lr.score(X_train,y_train)
test_score=lr.score(X_test,y_test)
coeff_used = np.sum(lr['classifier'].coef_!=0)

In [292]:
print(train_score)
print(test_score)
print(coeff_used)

0.9012842882417406
0.9034154704252428
51


In [293]:
#Adjusted R-squared
n = len(X_train)
p = coeff_used
train_score_adj = 1-((1-train_score)*((n-1)/(n-p-1)))
train_score_adj

0.8959624052611156

In [294]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [295]:
#MAPE
y_pred = lr.predict(X_test)
test_mape=mape(y_test,y_pred)
print("MAPE: ",test_mape)
print("MSE: ",mean_squared_error(y_test, y_pred, squared=False))

MAPE:  954.2818944388005
MSE:  577714.5828817701


In [296]:
coeffs = []
for i in range(0, len(feature_names)):
    coeffs.append((lr['classifier'].coef_[i],feature_names[i]))

## Retrain Lasso on Entire Data Set

In [297]:
lr.fit(X, y)

C:\Users\jdonv\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11947947834883.188, tolerance: 392872319013.2187
  positive)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['CAL_TPO_MONTH',
                                                   'CAL_WEEK_TYPE', 'SEASON',
                                                   'CATEGORY']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['LTO_ALTS', 'LTO_CARBONATED',
     

In [298]:
coeff_used = np.sum(lr['classifier'].coef_!=0)

In [299]:
coeffs = []
for i in range(0, len(feature_names)):
    coeffs.append((lr['classifier'].coef_[i],feature_names[i]))

In [5]:
#coeffs

In [301]:
lr['classifier'].intercept_

array([748608.4401989])

In [302]:
print("R^2: ",lr.score(X,y))

R^2:  0.9027020838057613


In [309]:
#Output coefficients to Excel
pd.DataFrame(coeffs, columns=['coefficients', 'features']).to_excel("Model_Results_Dec11-flv.xlsx")

In [311]:
#For Error analysis
#y_pred_all = lr.predict(X)
#pd.DataFrame(y_pred_all).to_csv("y_pred_all.csv")
#pd.DataFrame(X).to_csv('X.csv')
df.to_excel('Output-Flavours.xlsx')